In [ ]:
import os.path
import datetime
import warnings
warnings.filterwarnings('ignore')

import numpy as np
import netCDF4
from matplotlib import pyplot
from matplotlib import dates

setup_dir = "."
varname = "temp"

In [ ]:
# Read satellite observations of sea surface temperature
obs = []
if varname == 'temp':
    for l in open(os.path.join(setup_dir, 'cci_sst.dat')):
        if not l.startswith('#'):
            dt, value, sd = l.rstrip().split('\t')
            obs.append([datetime.datetime.strptime(dt, '%Y-%m-%d %H:%M:%S'), float(value), float(sd)])
obs = np.array(obs).reshape(-1, 3)

## Plot default GOTM results

Forecast-only, no data assimilation

In [ ]:
nc = netCDF4.Dataset(os.path.join(setup_dir, 'result.nc'))

time = netCDF4.num2date(nc['time'], nc['time'].units)
mpltime = dates.date2num(time)
z = nc['z'][:, :, 0, 0]
temp = nc[varname][:, :, 0, 0]
sst = temp[:, -1]

fig, (ax1, ax2) = pyplot.subplots(figsize=(8,8), nrows=2)

ax1.plot_date(obs[:,0], obs[:,1], '.', label='satellite')
ax1.plot_date(mpltime, sst, '-', label='model')
ax1.set_xlim(mpltime[0], mpltime[-1])
ax1.set_ylabel('temperature (\u00b0C)')
ax1.grid()
ax1.legend()
ax1.set_title('sea surface temperature')

mpltime_2d = np.broadcast_to(mpltime[:, np.newaxis], z.shape)
pc = ax2.contourf(mpltime_2d, z, temp)
cb = fig.colorbar(pc, ax=ax2)
cb.set_label('potential temperature (\u00b0C)')
ax2.set_ylabel('height (m)')
ax2.set_title('water temperature')
ax2.xaxis.axis_date()

## Plot ensemble GOTM results

In [ ]:
N = 4
ncs = [netCDF4.Dataset(os.path.join(setup_dir, 'result_%04i.nc' % (i + 1))) for i in range(N)]

enstime = netCDF4.num2date(ncs[0]['time'], ncs[0]['time'].units)
ensmpltime = dates.date2num(enstime)
z = nc.variables['z'][:, :, 0, 0]
temps = [nc[varname][:, :, 0, 0] for nc in ncs]
ssts = [temp[:, -1] for temp in temps]

fig, (ax1, ax2) = pyplot.subplots(figsize=(8,12), nrows=2)
ax1.plot_date(obs[:,0], obs[:,1], '.', label='satellite')
ax1.plot_date(time, sst, '-', label='model, no DA')
ax1.plot_date(enstime, np.mean(ssts, axis=0), '-', label='model, DA')
ax1.set_xlim(time[0], time[-1])
ax1.set_ylabel('temperature (\u00b0C)')
ax1.grid()
ax1.legend()
ax1.set_title('sea surface temperature')

mpltime_2d = np.broadcast_to(ensmpltime[:, np.newaxis], z.shape)
temp_diff = np.mean(temps, axis=0) - temp[-enstime.size:, :]
pc = ax2.contourf(mpltime_2d, z, temp_diff, cmap='RdBu_r', levels=np.linspace(-1.,1.,21), extend='both')
cb = fig.colorbar(pc, ax=ax2)
cb.set_label('temperature difference (\u00b0C)')
ax2.set_ylabel('height (m)')
ax2.set_title('water temperature difference (DA - no DA)');
ax2.xaxis.axis_date()